# Obtaining robust cell population markers, and redefining/reassuring the biased cell populations

**TO RUN THIS NOTEBOOK YOU HAVE TO RUN 3H NOTEBOOK FULLY!!!**

## imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import scipy.sparse as spr
import networkx as nx

In [ ]:
!pip install cellassign

In [ ]:
# local imports and imports from other notebooks
from cellassign import assign_cats
from fb_functions import make_gene_scoring_with_expr, plot_score_graph, plot_UMAPS_gene, plot_adata_cluster_properties, make_dicts_fraction_mean, plot_dotplot_gene
%store -r dict_colors_mouse
%store -r seed
%store -r magma
%store -r data_dir
%store -r dict_cats_fb_mouse

In [ ]:
mpl.rcParams['figure.dpi'] = 120
pd.options.display.float_format = "{:,.2f}".format

**IMPORTANT: I am running this analysis in a computer with ~500 GB of RAM. I will load many datasets at once, which might be too much for some computers. I took this decision conciously, to have as much info available at any time as possible. If you cannot run all the analysis at once, you can run it by parts.**

## Anndata loading

In [ ]:
abbasi_2020_dir = data_dir + '/abassi_2020'
abassi_2020_ctrl_mouse = sc.read(f"{abbasi_2020_dir}/abassi_2020_ctrl_mouse_processed.h5")
abassi_2020_ctrl_mouse_fb = sc.read(f"{abbasi_2020_dir}/abassi_2020_ctrl_mouse_fb_robust.h5")

In [ ]:
boothby_2021_dir = data_dir + '/boothby_2021'
boothby_2021_ctrl_mouse = sc.read(boothby_2021_dir + '/boothby_2021_ctrl_mouse_processed.h5')
boothby_2021_ctrl_mouse_fb = sc.read(boothby_2021_dir + '/boothby_2021_ctrl_mouse_fb_robust.h5')

In [ ]:
buechler_2021_dir = data_dir + '/buechler_2021'
buechler_2021_ctrl_mouse = sc.read(buechler_2021_dir + '/buechler_2021_ctrl_mouse_processed.h5')
buechler_2021_ctrl_mouse_fb = sc.read(buechler_2021_dir + '/buechler_2021_ctrl_mouse_fb_robust.h5')

In [ ]:
haensel_2021_dir = data_dir + '/haensel_2021'
haensel_2021_ctrl_mouse_wounding = sc.read(haensel_2021_dir + '/haensel_2021_ctrl_mouse_wounding_processed.h5')
haensel_2021_ctrl_mouse_wounding_fb = sc.read(haensel_2021_dir + '/haensel_2021_ctrl_mouse_wounding_fb_robust.h5')

In [ ]:
joost_2020_dir = data_dir + '/joost_2020'
joost_2020_ctrl_mouse = sc.read(joost_2020_dir + '/joost_2020_ctrl_mouse_processed.h5')
joost_2020_ctrl_mouse_fb = sc.read(joost_2020_dir + '/joost_2020_ctrl_mouse_fb_robust.h5')

In [ ]:
phan_2020_dir = data_dir + '/phan_2020'
phan_2020_ctrl_mouse_21d = sc.read(f"{phan_2020_dir}/phan_2020_ctrl_mouse_21d_processed.h5")
phan_2020_ctrl_mouse_21d_fb = sc.read(f"{phan_2020_dir}/phan_2020_ctrl_mouse_21d_fb_robust.h5")

In [ ]:
shin_2020_dir = data_dir + '/shin_2020'
shin_2020_ctrl_mouse = sc.read(f"{shin_2020_dir}/shin_2020_ctrl_mouse_processed.h5")
shin_2020_ctrl_mouse_fb = sc.read(f"{shin_2020_dir}/shin_2020_ctrl_mouse_fb_robust.h5")

In [ ]:
shook_2020_dir = data_dir + '/shook_2020'
shook_2020_ctrl_mouse = sc.read(f"{shook_2020_dir}/shook_2020_ctrl_mouse_processed.h5")
shook_2020_ctrl_mouse_fb = sc.read(f"{shook_2020_dir}/shook_2020_ctrl_mouse_fb_robust.h5")

In [ ]:
vorstandlechner_2021_dir = data_dir + '/Vorstandlechner_2021'
vorstandlechner_2021_ctrl_mouse = sc.read(f"{vorstandlechner_2021_dir}/vorstandlechner_2021_ctrl_mouse_processed.h5")
vorstandlechner_2021_ctrl_mouse_fb = sc.read(f"{vorstandlechner_2021_dir}/vorstandlechner_2021_ctrl_mouse_fb_robust.h5")

# Geting the definitive list of genes
This is not *really* definitive. We use the produced markers to later on, in the analysis table (not in notebooks) do a selection based on gene function, or pattern of expression. The *final* list is in the variable **genes** below.

In [ ]:
%store -r list_all_datasets_mouse
%store -r list_datasets_mouse

%store -r list_accepted_clusters_mouse
%store -r list_accepted_axis_mouse
%store -r list_names_mouse

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse = make_gene_scoring_with_expr(list_datasets=list_datasets_mouse, calculate_DEGs = True, group_name = 'cluster_robust', 
                                                     value_ref = 'scores', select_method = 'pval', 
                                                     list_clusters = list_accepted_clusters_mouse,
                                                     select_thres = 0.05)

In [ ]:
dict_make_gene_scoring_axis_robust_mouse = make_gene_scoring_with_expr(list_datasets=list_datasets_mouse, calculate_DEGs = True, group_name = 'axis_robust',
                                                     value_ref = 'scores', select_method = 'pval', select_thres = 0.05, 
                                                         list_clusters =list_accepted_axis_mouse,)

In [ ]:
%store dict_make_gene_scoring_cluster_robust_mouse
%store dict_make_gene_scoring_axis_robust_mouse

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['x1'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['x2'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['x/y'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['y1'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['y2'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['y3'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['y4'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['y5'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['z1'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['z2'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['w/x'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['w1'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['w2'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['w3'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['w4'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['w5'].iloc[:50].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_mouse['v1'].iloc[:50].sort_index()

## Analyse the presence and expression pattern of marker genes (within fibroblast populations)

In this section we will plot UMAPs and dot plots of expression patterns of gene markers. This is useful because many gene markers tend to be markers of few clusters based on DE analysis but, in reality, they are also expressed in other clusters which might need to be considered.

In [ ]:
N = 25
genes = ['1500009L16Rik', '2200002D01Rik', '2810417H13Rik', '4631405K08Rik', '6030408B16Rik', 'AI607873', 'Abca6', 'Abca8', 'Abca8a', 'Abca8b', 'Abca9', 'Abcc9', 'Abi3bp', 'Acan', 'Ace', 'Ackr2', 'Ackr3', 'Ackr4', 'Acta2', 'Actn1', 'Adam12', 'Adam23', 'Adamts1', 'Adamts18', 'Adamts2', 'Adamts4', 'Adamts5', 'Adamts6', 'Adamts9', 'Adamtsl1', 'Adamtsl3', 'Adamtsl4', 'Adcy1', 'Adcyap1r1', 'Adgrd1', 'Adgrg2', 'Adora2b', 'Aebp1', 'Afap1l2', 'Agt', 'Agtr1b', 'Ahrr', 'Aif1l', 'Akr1c18', 'Akr1cl', 'Aldh1a3', 'Aldh3a1', 'Alpl', 'Alx4', 'Amigo2', 'Angpt4', 'Angptl1', 'Angptl4', 'Angptl7', 'Anln', 'Anxa1', 'Anxa3', 'Aoc3', 'Apbb1ip', 'Apcdd1', 'Apela', 'Aplp1', 'Apoc1', 'Apod', 'Apoe', 'Aqp1', 'Aqp3', 'Arhgap28', 'Arhgdib', 'Arid5a', 'Arl4a', 'Arl5b', 'Arpc1b', 'Aspn', 'Atf3', 'Atp10a', 'Atp1b1', 'Avpr1a', 'Axin2', 'Axl', 'Basp1', 'Bcam', 'Bcl11a', 'Bcl11b', 'Bcl2', 'Bcl3', 'Bgn', 'Bhlhe40', 'Bhlhe41', 'Birc3', 'Birc5', 'Bmp1', 'Bmp4', 'Bmp5', 'Bmp7', 'Bmper', 'Bnc2', 'Boc', 'Bok', 'Btbd11', 'Btg1', 'Btg2', 'C1qtnf3', 'C1qtnf6', 'C1s', 'C1s1', 'C2', 'C3', 'C4b', 'C6', 'C7', 'Cadm1', 'Cadm3', 'Cald1', 'Camk2a', 'Capn6', 'Casp4', 'Casz1', 'Cav1', 'Cav2', 'Cavin2', 'Cbfa2t3', 'Ccbe1', 'Ccdc3', 'Ccdc80', 'Cck', 'Ccl11', 'Ccl2', 'Ccl7', 'Ccl8', 'Ccl9', 'Ccn2', 'Ccn5', 'Ccna2', 'Ccnb1', 'Ccnb2', 'Ccnl1', 'Cd109', 'Cd151', 'Cd200', 'Cd248', 'Cd24a', 'Cd34', 'Cd36', 'Cd55', 'Cd81', 'Cd9', 'Cdc20', 'Cdca3', 'Cdca8', 'Cdh11', 'Cdh13', 'Cdh3', 'Cdh4', 'Cdkn2b', 'Cdkn3', 'Cdon', 'Cebpb', 'Cenpa', 'Cenpe', 'Cenpf', 'Cenpm', 'Cenpw', 'Cfh', 'Cfl1', 'Cgref1', 'Chodl', 'Chpf', 'Chrdl1', 'Chst1', 'Chst15', 'Chst8', 'Cib3', 'Cilp', 'Ckap2', 'Ckap2l', 'Cks2', 'Cldn1', 'Cldn10', 'Clec11a', 'Clec14a', 'Clec3b', 'Clu', 'Cnn2', 'Cnn3', 'Cntn1', 'Coch', 'Col11a1', 'Col12a1', 'Col13a1', 'Col14a1', 'Col15a1', 'Col16a1', 'Col1a1', 'Col1a2', 'Col23a1', 'Col27a1', 'Col3a1', 'Col4a1', 'Col4a2', 'Col4a4', 'Col5a1', 'Col5a2', 'Col5a3', 'Col6a1', 'Col6a2', 'Col6a3', 'Col6a6', 'Col7a1', 'Col8a1', 'Col8a2', 'Copz2', 'Corin', 'Cox4i2', 'Cox6b2', 'Cp', 'Cpe', 'Cpne5', 'Cpxm2', 'Cpz', 'Crabp1', 'Crabp2', 'Creb3l1', 'Creb3l3', 'Creb5', 'Crip2', 'Crp', 'Csf1', 'Csf1r', 'Cspg4', 'Csrnp1', 'Csrp1', 'Ctbp2', 'Cthrc1', 'Ctnnal1', 'Ctsb', 'Ctsk', 'Cttnbp2', 'Ctxn1', 'Cxadr', 'Cxcl1', 'Cxcl10', 'Cxcl12', 'Cxcl14', 'Cxcl16', 'Cxcl2', 'Cxcr4', 'Cybrd1', 'Cygb', 'Cyp1b1', 'Cyp26b1', 'Cyp2f2', 'Cyp4b1', 'Cyp7b1', 'Daam2', 'Dact1', 'Dbn1', 'Dcbld2', 'Dcn', 'Ddah2', 'Ddit4', 'Deptor', 'Dhrs3', 'Diaph3', 'Dkk2', 'Dkk3', 'Dll1', 'Dmd', 'Dock9', 'Dpp4', 'Dsc3', 'Dusp10', 'Dusp5', 'Ebf1', 'Ebf2', 'Ecm1', 'Ecm2', 'Edn3', 'Ednra', 'Ednrb', 'Eef1a1', 'Efemp1', 'Efhd1', 'Efna1', 'Efnb1', 'Egfl6', 'Egflam', 'Egfr', 'Egr1', 'Egr3', 'Ell2', 'Eln', 'Emid1', 'Emilin2', 'Emp2', 'Emx2', 'Endod1', 'Enho', 'Enpp2', 'Enpp3', 'Entpd1', 'Entpd2', 'Epha3', 'Eps8', 'Errfi1', 'Etl4', 'Ets2', 'Etv1', 'Etv4', 'Eva1a', 'Ezr', 'F11r', 'F13a1', 'F2r', 'F3', 'F5', 'Fabp4', 'Fabp5', 'Fam13a', 'Fam180a', 'Fam198b', 'Fam64a', 'Fap', 'Fbln1', 'Fbln2', 'Fbln7', 'Fbn1', 'Fbxo32', 'Fcgr3', 'Fermt1', 'Fgf7', 'Fgf9', 'Fgfr2', 'Fgfr4', 'Fgl2', 'Fhl2', 'Fibin', 'Figf', 'Fkbp9', 'Flnc', 'Fmo1', 'Fmo2', 'Fmod', 'Fnbp1l', 'Fndc1', 'Fosb', 'Fosl1', 'Foxd2', 'Foxp1', 'Foxs1', 'Frmd4b', 'Frmd6', 'Fst', 'Fstl1', 'Fxyd3', 'Fxyd6', 'Fzd1', 'Fzd4', 'G0s2', 'Gab1', 'Gadd45b', 'Galnt15', 'Gap43', 'Gas1', 'Gas6', 'Gbp5', 'Gch1', 'Gdf10', 'Gdf15', 'Gfpt2', 'Gfra1', 'Ggt5', 'Ghr', 'Gldn', 'Glt8d2', 'Gm12481', 'Gm12840', 'Gm14603', 'Gm1673', 'Gm17455', 'Gm20186', 'Gm45551', 'Gm48159', 'Gm48942', 'Gm5084', 'Gm973', 'Gng13', 'Gngt2', 'Gpc1', 'Gpc3', 'Gpc4', 'Gpha2', 'Gpm6b', 'Gpnmb', 'Gpx3', 'Greb1l', 'Grem1', 'Grem2', 'Gsn', 'H2afz', 'Has1', 'Has2', 'Heg1', 'Hey2', 'Hgf', 'Hhip', 'Hic1', 'Hmcn1', 'Hmcn2', 'Hmga1', 'Hmgb2', 'Hmmr', 'Hmox1', 'Hoxd9', 'Hpgd', 'Hs3st6', 'Hspg2', 'Htra1', 'Htra3', 'Icam1', 'Id1', 'Id4', 'Ier3', 'Ier5', 'Ifi204', 'Ifi205', 'Ifi206', 'Ifi27l2a', 'Ifrd1', 'Igf1', 'Igfbp2', 'Igfbp3', 'Igfbp4', 'Igfbp5', 'Igfbp6', 'Igfbp7', 'Igsf10', 'Igsf9', 'Il11ra', 'Il11ra1', 'Il18', 'Il33', 'Il6', 'Inhba', 'Irak3', 'Irf1', 'Irf8', 'Islr', 'Islr2', 'Ism1', 'Isyna1', 'Itga6', 'Itgb1', 'Itgb4', 'Itgb7', 'Itgbl1', 'Itih5', 'Itm2a', 'Itm2b', 'Itpkc', 'Jag1', 'Junb', 'Kcnj8', 'Kcnk2', 'Kctd1', 'Kdelr3', 'Kdm6b', 'Kera', 'Kiaa1217', 'Kif26b', 'Klf5', 'Knstrn', 'Krt19', 'Ktn1', 'Lamb3', 'Lamc3', 'Ldb2', 'Lef1', 'Lepr', 'Lgals3bp', 'Lgmn', 'Lgr5', 'Lhx2', 'Lif', 'Limch1', 'Lims2', 'Lmo7', 'Lockd', 'Lox', 'Loxl1', 'Loxl2', 'Loxl4', 'Lpl', 'Lrrc15', 'Lrrn4cl', 'Lsamp', 'Lsp1', 'Ltbp2', 'Ltbp4', 'Lum', 'Lxn', 'Ly6c1', 'Mafb', 'Maff', 'Mamdc2', 'Marcksl1', 'Marveld1', 'Masp1', 'Matn2', 'Mdfic', 'Mdk', 'Medag', 'Mef2c', 'Megf6', 'Meis2', 'Meox1', 'Meox2', 'Mest', 'Metrnl', 'Mfap2', 'Mfap4', 'Mfap5', 'Mgll', 'Mgp', 'Mgst1', 'Mgst3', 'Miat', 'Mical2', 'Mki67', 'Mkx', 'Mme', 'Mmp16', 'Mmp2', 'Mmp27', 'Mmp3', 'Mnda', 'Moxd1', 'Mpped2', 'Mrps6', 'Msx1', 'Mt1', 'Mt2', 'Mtcl1', 'Mtss1', 'Mtus1', 'Mup20', 'Mxd3', 'Mxra8', 'Myc', 'Myh9', 'Mylk', 'Myo1b', 'Myoc', 'Naalad2', 'Nav2', 'Nbl1', 'Ncam1', 'Ndnf', 'Ndp', 'Ndrg1', 'Ndrg2', 'Ndufa4l2', 'Nfatc2', 'Nfe2l2', 'Nfib', 'Nfkb1', 'Nfkbia', 'Nfkbiz', 'Ngfr', 'Nhsl1', 'Nid1', 'Nid2', 'Nkain4', 'Nkd1', 'Nkd2', 'Nnmt', 'Noct', 'Notum', 'Nova1', 'Npm1', 'Npnt', 'Npr1', 'Npy1r', 'Nr1h3', 'Nr2f1', 'Nr2f2', 'Nr4a1', 'Nr4a3', 'Nradd', 'Nrep', 'Nrg2', 'Nrp1', 'Nrp2', 'Nt5e', 'Ntrk2', 'Ntrk3', 'Nuak1', 'Nucb2', 'Nupr1', 'Nxn', 'Ogn', 'Olfml2a', 'Olfml2b', 'Omd', 'Osmr', 'P2ry14', 'P4ha2', 'Palld', 'Palmd', 'Pam', 'Pamr1', 'Pappa', 'Pappa2', 'Paqr6', 'Pard6g', 'Pawr', 'Pcolce', 'Pcolce2', 'Pcsk5', 'Pcsk6', 'Pde3a', 'Pdgfrl', 'Pdrg1', 'Pear1', 'Peg3', 'Perp', 'Pfn2', 'Pgm2l1', 'Phlda1', 'Phlda3', 'Pi16', 'Pianp', 'Pim1', 'Pkdcc', 'Pkp3', 'Pla1a', 'Pla2g5', 'Plac8', 'Plat', 'Plcb1', 'Plcxd3', 'Plekha4', 'Plpp1', 'Plscr1', 'Plscr4', 'Pltp', 'Plxdc1', 'Pmepa1', 'Pnp2', 'Pnrc1', 'Podn', 'Postn', 'Pparg', 'Ppib', 'Ppic', 'Ppl', 'Ppp1r14a', 'Ppp1r14b', 'Prdm1', 'Prelp', 'Prex1', 'Prex2', 'Prkg2', 'Prlr', 'Procr', 'Prr5l', 'Prss12', 'Prss23', 'Ptch1', 'Ptch2', 'Ptger3', 'Ptgfr', 'Ptgis', 'Ptgs1', 'Ptgs2', 'Pth1r', 'Ptk7', 'Ptma', 'Ptprd', 'Ptpre', 'Ptprf', 'Ptprk', 'Ptx3', 'Pxn', 'Pycr1', 'Qpct', 'Rab32', 'Rab34', 'Racgap1', 'Ramp2', 'Ramp3', 'Rarres1', 'Rarres2', 'Rasd1', 'Rasgrp2', 'Rasl11b', 'Rassf4', 'Rassf9', 'Rbp1', 'Rbp4', 'Rcn3', 'Rel', 'Ret', 'Rflnb', 'Rgs2', 'Rnf122', 'Rnf152', 'Robo1', 'Robo2', 'Rrbp1', 'Rspo1', 'Rspo3', 'Rspo4', 'Rtn4r', 'Runx2', 'Runx3', 'S100a4', 'S100b', 'Sapcd1', 'Satb2', 'Sbsn', 'Sbspon', 'Scara3', 'Scara5', 'Scarf2', 'Scg3', 'Scn7a', 'Scube3', 'Sdc1', 'Sdc3', 'Sdk1', 'Sectm1a', 'Sema3a', 'Sema3b', 'Sema3c', 'Sema3d', 'Sema3e', 'Sema5a', 'Sepp1', 'Sept11', 'Sept4', 'Serpina3g', 'Serpina3n', 'Serpine2', 'Serpinf1', 'Serping1', 'Serpinh1', 'Sesn3', 'Sfrp1', 'Sfrp2', 'Sfrp4', 'Shox2', 'Slc10a6', 'Slc12a2', 'Slc16a3', 'Slc26a7', 'Slc2a1', 'Slc38a10', 'Slc40a1', 'Slc48a1', 'Slc6a2', 'Slco2b1', 'Smc2', 'Smc4', 'Smim3', 'Smoc2', 'Smox', 'Smpd3', 'Sms-ps', 'Smurf2', 'Sned1', 'Socs3', 'Sod2', 'Sorbs1', 'Sorcs2', 'Sostdc1', 'Sox4', 'Sox9', 'Sparc', 'Sparcl1', 'Spc24', 'Spc25', 'Sphk1', 'Spint2', 'Spon1', 'Spon2', 'Spp1', 'Spry1', 'Srpx', 'Srpx2', 'Ssc5d', 'Stc1', 'Steap4', 'Stmn1', 'Stmn2', 'Sulf2', 'Sult5a1', 'Susd2', 'Svep1', 'Syne2', 'Syt13', 'Tacc3', 'Tagln', 'Tbx15', 'Tbxa2r', 'Tcf4', 'Tek', 'Tenm2', 'Tenm3', 'Tfap2a', 'Tfap2c', 'Tgfb2', 'Tgfbi', 'Tgfbr2', 'Tgm2', 'Thbd', 'Thbs1', 'Thbs2', 'Thbs3', 'Thbs4', 'Thsd4', 'Thy1', 'Tiam1', 'Timp2', 'Timp3', 'Tiparp', 'Tjp1', 'Tjp2', 'Tm4sf1', 'Tmeff2', 'Tmem100', 'Tmem119', 'Tmem132c', 'Tmem135', 'Tmem150c', 'Tmem176a', 'Tmem176b', 'Tmem204', 'Tnfaip2', 'Tnfaip3', 'Tnfaip6', 'Tnfrsf19', 'Tnfsf13b', 'Tnmd', 'Tnn', 'Tns3', 'Tnxb', 'Tpbg', 'Tpd52', 'Tpd52l1', 'Tpm2', 'Tpx2', 'Trabd2b', 'Trf', 'Trib1', 'Trib2', 'Tril', 'Trim29', 'Trim47', 'Trio', 'Trp63', 'Trpm3', 'Trps1', 'Tshz2', 'Tshz3', 'Tspan11', 'Tspan4', 'Tspan7', 'Tuba4a', 'Tubb4a', 'Twist2', 'Uap1', 'Uchl1', 'Ugcg', 'Ugdh', 'Vasn', 'Vcam1', 'Vcan', 'Vegfa', 'Vgll3', 'Vit', 'Vkorc1', 'Vtn', 'Vwa1', 'Wfdc1', 'Wif1', 'Wnt10a', 'Wnt10b', 'Wnt2', 'Wnt5a', 'Wnt6', 'Zc3h12a', 'Zfhx4', 'Zfp36', 'Zfp385a', 'Zfp385b', 'Zfp608', 'Zim1', 'Znf608', 'Zyx', 'a']

To create the dot plot we will use two variables: the percentage of cells expressing the marker in each cluster, and the mean expression value of the expressing-cells in each cluster. To do that, we will create a df for each case and gene, compile the info for each adata and get the mean/std across the adatas. If one gene is not expressed in an adata, or the adata does not have a certain cluster, the value registered is a NaN, not zero. 

In [ ]:
plot_UMAPS_gene('cluster_robust', list_datasets=list_all_datasets_mouse, list_names=list_names_mouse, n_cols=5)

In [ ]:
dict_fraction_cells, dict_mean_exp = make_dicts_fraction_mean(genes, list_all_datasets=list_all_datasets_mouse, list_accepted_clusters=list_accepted_clusters_mouse, 
                                                              clusterby='cluster_robust', list_names=list_names_mouse)

To do the dotplot we do some adjustments. We adjust the fraction of cells beign expressed to $x^{2/3}$ to make it slightly more visual. We also apply a alpha of 1 - std(fraction of cells) to account for markers that are expressed in disparily, so that they are visually not that relevant.


In [ ]:
for gene in genes:
    print(gene)
    plot_dotplot_gene(gene, dict_fraction_cells, dict_mean_exp)
    plot_UMAPS_gene(gene, list_datasets=list_all_datasets_mouse, list_names=list_names_mouse, n_cols=5)
    plt.show()

## Analyse the presence and expression pattern of marker genes (within all populations)

In this section we will plot UMAPs and dot plots of expression patterns of gene markers. This is useful because many gene markers tend to be markers of few clusters based on DE analysis but, in reality, they are also expressed in other clusters which might need to be considered.

In [ ]:
list_accepted_clusters_mouse_full = list(dict_cats_fb_mouse.keys())

In [ ]:
list_all_datasets_mouse_full = [abassi_2020_ctrl_mouse, boothby_2021_ctrl_mouse, 
                         buechler_2021_ctrl_mouse, joost_2020_ctrl_mouse, phan_2020_ctrl_mouse_21d, shook_2020_ctrl_mouse, 
                         vorstandlechner_2021_ctrl_mouse, ]

list_names_mouse_full = [adata.obs['Author'].values[0] + ' ' + str(int(adata.obs['Year'].values[0])) + ' mouse' for adata in list_all_datasets_mouse_full]

In [ ]:
plot_UMAPS_gene('assigned_cats', list_datasets=list_all_datasets_mouse_full, list_names=list_names_mouse_full, n_cols=3)

In [ ]:
dict_fraction_cells_full, dict_mean_exp_full = make_dicts_fraction_mean(genes, list_all_datasets=list_all_datasets_mouse_full, list_names=list_names_mouse_full,
                                                                        list_accepted_clusters=list_accepted_clusters_mouse_full, clusterby='assigned_cats')

In [ ]:
for gene in genes:
    print(gene)
    plot_dotplot_gene(gene, dict_fraction_cells_full, dict_mean_exp_full, rotate=True)
    plot_UMAPS_gene(gene, list_datasets=list_all_datasets_mouse_full, list_names=list_names_mouse_full, n_cols=5)
    plt.show()

In [ ]:
plot_UMAPS_gene('cluster_robust', list_datasets=list_all_datasets_mouse, list_names=list_names_mouse_full, n_cols=3)

In [ ]:
gene='Dcn'
plot_UMAPS_gene(gene, list_datasets=list_all_datasets_mouse, list_names=list_names_mouse_full, n_cols=5)
